# Putting Files in the Cloud!

In [2]:
import boto3
import pandas as pd

In [3]:
# check if boto3 is installed
boto3.__version__

'1.26.15'

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
#environment variables
ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")



In [65]:
s3= boto3.client('s3',
                 region_name='eu-central-1',
                 aws_access_key_id=f'{ACCESS_KEY_ID}',
                 aws_secret_access_key=f'{AWS_SECRET_KEY}')

In [6]:
respose = s3.list_buckets()
for bucket in respose['Buckets']:
    print(bucket)

{'Name': 'my-first-bucket-128341164379', 'CreationDate': datetime.datetime(2022, 11, 24, 14, 2, 29, tzinfo=tzutc())}
{'Name': 'samil-aws-config', 'CreationDate': datetime.datetime(2022, 11, 24, 11, 11, 58, tzinfo=tzutc())}


In [ ]:
bucket = s3.create_bucket(Bucket='my-first-bucket-128341164379',
                          CreateBucketConfiguration={'LocationConstraint': 'eu-central-1'})

- Get the buckets from S3.
- Iterate over the bucket key from response to access the list of buckets.
- Print the name of each bucket.

In [7]:
response = s3.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'], bucket['CreationDate'])

my-first-bucket-128341164379 2022-11-24 14:02:29+00:00
samil-aws-config 2022-11-24 11:11:58+00:00


In [70]:
response = s3.delete_bucket(Bucket='my-first-bucket-128341164379')

In [9]:
# Delete all the buckets with 'gim', create replacements.
for bucket in response['Buckets']:
  if 'gim' in bucket['Name']:
      s3.delete_bucket(Bucket=bucket['Name'])

In [13]:
# Upload a new file.
# Filename is the local file path.
# Bucket is the name of the bucket.
# Key is what we want to call the object in S3.
s3.upload_file(Filename= '/Users/ssamilozkan/Desktop/aws_boto/dataset/airports.csv' ,
               Bucket= 'my-first-bucket-128341164379' , 
               Key= 'aiport.csv' )

In [14]:
# List all the objects in the bucket.
response = s3.list_objects(Bucket= 'my-first-bucket-128341164379', MaxKeys=2)
for obj in response['Contents']:
    print(obj['Key'], obj['Size'], obj['LastModified'])

aiport.csv 84548 2022-11-25 15:26:52+00:00
planes.csv 219326 2022-11-25 14:21:22+00:00


In [15]:
# Download the file.
# You can see the new file in the current directory.
s3.download_file(Filename= '/Users/ssamilozkan/Desktop/aws_boto/dataset/airports2.csv' ,
                 Bucket = 'my-first-bucket-128341164379' ,
                 Key = 'aiport.csv' )

In [ ]:
# Deleteing objects
s3.delete_object(Bucket= 'my-first-bucket-128341164379' , Key= 'aiport.csv' )

In [17]:
# object metadata
s3.head_object(Bucket= 'my-first-bucket-128341164379' , Key= 'aiport.csv' )

{'ResponseMetadata': {'RequestId': 'XKMSR3GCZ2R03XZA',
  'HostId': 'owokeTx7VACJAehcJjyjvBJ4rjkK3pmu6OMHvKc3q9tM9lTe4Kfyrdv7/U7tTlQso1AiQciT+3I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'owokeTx7VACJAehcJjyjvBJ4rjkK3pmu6OMHvKc3q9tM9lTe4Kfyrdv7/U7tTlQso1AiQciT+3I=',
   'x-amz-request-id': 'XKMSR3GCZ2R03XZA',
   'date': 'Fri, 25 Nov 2022 13:53:42 GMT',
   'last-modified': 'Fri, 25 Nov 2022 13:53:36 GMT',
   'etag': '"a31ea87f5412c40ed4907c38478e5c80"',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'server': 'AmazonS3',
   'content-length': '84548'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2022, 11, 25, 13, 53, 36, tzinfo=tzutc()),
 'ContentLength': 84548,
 'ETag': '"a31ea87f5412c40ed4907c38478e5c80"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {}}

In [ ]:
# List only objects that start with '2018/final_'
response = s3.list_objects(Bucket='gid-staging', 
                           Prefix='2018/final_')

# Iterate over the objects
if 'Contents' in response:
  for obj in response['Contents']:
      # Delete the object
      s3.delete_object(Bucket='gid-staging', Key=obj['Key'])

# Print the remaining objects in the bucket
response = s3.list_objects(Bucket='gid-staging')

for obj in response['Contents']:
  	print(obj['Key'])

In [16]:
#ACL (Access Control List)
# set the ACL to public-read
# Now anyone can read the object
s3.put_object_acl(Bucket='my-first-bucket-128341164379', Key = 'aiport.csv', ACL='public-read')

{'ResponseMetadata': {'RequestId': '91F81RTDBAKKE1CH',
  'HostId': 'VV5C77tdB9cs3XRJrLmeyR3mIuHmHXUvzYVk3snHwjwFYR8lvx00opAVnqamTbXbydVVGjLB8ek=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'VV5C77tdB9cs3XRJrLmeyR3mIuHmHXUvzYVk3snHwjwFYR8lvx00opAVnqamTbXbydVVGjLB8ek=',
   'x-amz-request-id': '91F81RTDBAKKE1CH',
   'date': 'Fri, 25 Nov 2022 15:27:15 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [17]:
# Setting the ACLs on upload
# upload file with public-read ACL
s3.upload_file(Filename= '/Users/ssamilozkan/Desktop/aws_boto/dataset/planes.csv' ,
               Bucket= 'my-first-bucket-128341164379' ,
               Key= 'planes.csv' ,
               ExtraArgs={'ACL': 'public-read'})

-  **Accessing the public objects**
- URL for Key = `https://{bucket}.s3.{region}.amazonaws.com/{key}` 

In [18]:
# Generating public object URL
url = "https://{}.s3.{}.amazonaws.com/{}".format('my-first-bucket-128341164379', 'eu-central-1','planes.csv')

In [19]:
# read the URL into a pandas dataframe
df = pd.read_csv(url)
df.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
1,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N105UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N107US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan


In [11]:
# Set the ACL to private
s3.put_object_acl(Bucket = 'my-first-bucket-128341164379', Key = 'planes.csv', ACL='private')

{'ResponseMetadata': {'RequestId': '1ZXGV8693THA3PES',
  'HostId': 'NtaJXDawSroYmLJsTeOBEPBclVKLPtf/PXM+HhnOxphGKNIAxKg5s0ngw8SRnI5apAbL5XMCHJA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NtaJXDawSroYmLJsTeOBEPBclVKLPtf/PXM+HhnOxphGKNIAxKg5s0ngw8SRnI5apAbL5XMCHJA=',
   'x-amz-request-id': '1ZXGV8693THA3PES',
   'date': 'Sun, 27 Nov 2022 15:39:59 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [9]:
# List only objects that start with 'p'
response = s3.list_objects(
    Bucket='my-first-bucket-128341164379', Prefix='p')

# Iterate over the objects
for obj in response['Contents']:
  
    # Give each object ACL of public-read
    s3.put_object_acl(Bucket='my-first-bucket-128341164379', 
                      Key=obj['Key'], 
                      ACL='public-read')
    
    # Print the Public Object URL for each object
    print("https://{}.s3.amazonaws.com/{}".format('my-first-bucket-128341164379', obj['Key']))

https://my-first-bucket-128341164379.s3.amazonaws.com/planes.csv


In [13]:
# Accessing private objects
obj = s3.get_object(Bucket='my-first-bucket-128341164379', Key='planes.csv')
# print(obj['Body'].read())
print(obj)

{'ResponseMetadata': {'RequestId': '85E743HSCS72P3NY', 'HostId': 'BvaBfgHEPgNpbFnNAx5eE9s+et0ww4JxLoNJLSepapl4rgj48ZWDdL7/o5m5EnIF80KFETFVzWc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'BvaBfgHEPgNpbFnNAx5eE9s+et0ww4JxLoNJLSepapl4rgj48ZWDdL7/o5m5EnIF80KFETFVzWc=', 'x-amz-request-id': '85E743HSCS72P3NY', 'date': 'Sun, 27 Nov 2022 15:41:09 GMT', 'last-modified': 'Fri, 25 Nov 2022 15:27:19 GMT', 'etag': '"42949138987d62284b36668386cefb76"', 'accept-ranges': 'bytes', 'content-type': 'binary/octet-stream', 'server': 'AmazonS3', 'content-length': '219326'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2022, 11, 25, 15, 27, 19, tzinfo=tzutc()), 'ContentLength': 219326, 'ETag': '"42949138987d62284b36668386cefb76"', 'ContentType': 'binary/octet-stream', 'Metadata': {}, 'Body': <botocore.response.StreamingBody object at 0x11fcf4eb0>}


In [14]:
# Read StramingBody in into a pandas dataframe
# 'Body': <botocore.response.StreamingBody object at 0x11fcf4eb0>
pd.read_csv(obj['Body'])


,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
1,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N105UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N107US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
...,...,...,...,...,...,...,...,...,...
2623,N983SW,2004.0,Fixed wing multi engine,BOMBARDIER INC,CL-600-2B19,2,55,NaN,Turbo-fan
2624,N984CA,1997.0,Fixed wing multi engine,CANADAIR,CL-600-2B19,2,55,NaN,Turbo-fan
2625,N986CA,NaN,Fixed wing multi engine,CANADAIR,CL-600-2B19,2,55,NaN,Turbo-fan
2626,N986SW,2004.0,Fixed wing multi engine,BOMBARDIER INC,CL-600-2B19,2,55,NaN,Turbo-fan


In [17]:
# Pre-signed URL
# Generate a presigned URL for the object
# We generate a presigned URL that grants access to the file for 1 hour or 3600 seconds
shared_url = s3.generate_presigned_url(ClientMethod='get_object',
                                       ExpiresIn=3600, 
                                       Params={'Bucket': 'my-first-bucket-128341164379', 'Key': 'planes.csv'})

# Read the URL into a pandas dataframe
pd.read_csv(shared_url).head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
1,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N105UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N107US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan


In [ ]:
## EXAMPLE -> dont run just for example
#For each file in response, load the object from S3.
#Load the object's StreamingBody into pandas, and append to df_list.
#Concatenate all the DataFrames with pandas.
#Preview the resulting DataFrame.#

df_list =  [ ] 

for file in response['Contents']:
    # For each file in response load the object from S3
    obj = s3.get_object(Bucket='gid-requests', Key=file['Key'])
    # Load the object's StreamingBody with pandas
    obj_df = pd.read_csv(obj['Body'])
    # Append the resulting DataFrame to list
    df_list.append(obj_df)

# Concat all the DataFrames with pandas
df = pd.concat(df_list)

# Preview the resulting DataFrame
df.head()


In [ ]:
# Many python data science tools have the ability to generate simple HTML from data
# The goal is not to have us making beautiful websiites, but to share our analysis with the world

In [ ]:
# HTML table in Pandas
# Convert DAtaFrame to HTML
# render_links=True will convert URLs to clickable links
# We can use the columns parameter to pass a list of only columns we want rendered
# boreder=0 will remove the border from the table(1 for border)
df.to_html('table.html', render_links=True, columns=['name', 'url'], border=0)

In [ ]:
# Uploading HTML file to S3
# we set the content type to text/html, the browser will know to render the file as HTML
s3.upload_file(Filename='table.html',
               Bucket='my-first-bucket-128341164379', 
               Key='table.html',
               ExtraArgs={'ContentType': 'text/html', 'ACL': 'public-read'})

In [ ]:
# Upload an image to S3
s3.upload_file(Filename='image.png',
               Bucket='my-first-bucket-128341164379', 
               Key='image.png', ExtraArgs={'ContentType': 'image/png', 'ACL': 'private'})

In [33]:
# Upload IANA media types to S3
# IANA media types are a list of all the different types of files that can be on the internet(JSON, HTML, PNG, PDF, etc.)
# Generate an index page
r = s3.list_objects(Bucket='my-first-bucket-128341164379', Prefix='p')

#Convert the response to a DataFrame
objects_df = pd.DataFrame(r['Contents'])
objects_df.head()
#r

,Key,LastModified,ETag,Size,StorageClass,Owner
0,planes.csv,2022-11-25 15:27:19+00:00,"""42949138987d62284b36668386cefb76""",219326,STANDARD,{'ID': '433f8c3ff6d834c99c6a33e91277f031d89d8a...


In [34]:
# We create a column 'Link' where we create the URL for every key in the object.
base_url = "https://{}.s3.amazonaws.com".format('my-first-bucket-128341164379')
objects_df['Link'] = base_url + '/' + objects_df['Key']

In [41]:
# Then we convert the DataFrame to HTML
objects_df.to_html('report.html', render_links=True, columns=['Key', 'Link'], border=1)


In [44]:
# Upload the HTML file to S3
# It will be public, so anyone can see it

s3.upload_file(Filename='report.html',
               Bucket='my-first-bucket-128341164379', 
               Key='report.html', 
               ExtraArgs={'ContentType': 'text/html', 'ACL': 'public-read'})
# https://my-first-bucket-128341164379.s3.eu-central-1.amazonaws.com/report.html

### The Steps
#### Prepare the data
1. Download files for month from the raw data bucket.
2. Concatenate them into one csv.
3. Create an aggregated DataFrame
#### Create the report 
1. Write a DataFrame to CSV and HTML.
2. Generate a Bokeh plot, save as HTML.
#### Upload report to sharable website
1. Create `reports` bucket.
2. Upload all three files for the month to S3.
3. Generate an index.html file. that lists all the files.
4. Get the website URL!


*Raw Data Bucket*
1. Private files
2. Daily csvs of request from App
3. Raw data
4. This bucket contains daily CSVs
5. Every CSV has rows for every request that was made in the GetItDone app that day.

In [ ]:
# Create list to hold our DataFrames
df = list = [ ]

# Request the list of objects in the bucket with the prefix 'p'
response = s3.list_objects(Bucket='my-first-bucket-128341164379', Prefix='p')

# Get response contents
request_files = response['Contents']

In [ ]:
# Iterate over each object
# we will iterate over each object, and load its StreamingBody using get_object.
# We can't use the Object URL because it's private
for file in request_files:
    obj = s3.get_object(Bucket='my-first-bucket-128341164379', Key=file['Key'])
    
    # Read it as DataFrame
    obj_df= pd.read_csv(obj['Body'])
    
    # Append to list
    # So we will have a list of 31 dataframes, one for each dat in the df_list variable
    df_list.append(obj_df)

In [ ]:
# Concatenate all the DataFrames in the list
df = pd.concat(df_list)

# Preview the resulting DataFrame
df.head()

In [ ]:
#Create aggregated reports
# to csv
df.to_csv('jan_final_report.csv')
# to html
df.to_html('jan_final_report.html', render_links=True, columns=['name', 'url'], border=0)


In [ ]:
#Upload Aggregated CSV to S3
s3.upload_file(Filename='jan_final_report.csv', 
               Key = '2022/jan/jan_final_report.csv', 
               Bucket='my-first-bucket-128341164379', 
               ExtraArgs={'ContentType': 'text/csv', 'ACL': 'public-read'})

In [ ]:
# Upload Aggregated HTML table to S3
s3.upload_file(Filename='jan_final_report.html',
               Key = '2022/jan/jan_final_report.html',
               Bucket='my-first-bucket-128341164379',
               ExtraArgs={'ContentType': 'text/html', 'ACL': 'public-read'})

In [ ]:
# Create index.html
# List the gid-reports bucket objects starting with 2019/
r = s3.list_objects(Bucket='my-first-bucket-128341164379', Prefix='2022/')
# Convert the response to a DataFrame
obj_df = pd.DataFrame(r['Contents'])
# Finally, we create a column 'Link' where we create the URL for every key in the object.
base_url = "https://{}.s3.amazonaws.com".format('my-first-bucket-128341164379')
obj_df['Link'] = base_url + '/' + obj_df['Key']

In [ ]:
# Write DataFrame to HTML
objects_df.to_html('report_listing.html',
                   render_links=True, 
                   columns=['Key', 'Link', 'Size', 'LastModified'], 
                   border=1)

In [ ]:
# Upload the file to my-first-bucket-128341164379 bucket root
s3.upload_file(Filename='report_listing.html', 
               Key = 'report_listing.html',
               Bucket='my-first-bucket-128341164379', 
               ExtraArgs={'ContentType': 'text/html', 
                          'ACL': 'public-read'})
# Get the URL of the index!

# Reporting and Notifying!

- Alerting humans and machines to take action is a key component of data engineering.That's where Amazon SNS comes in!

- *AWS SNS:* Stand for Simple Notification Service.

- SNS is a fully managed service that allows you to send messages to a variety of endpoints, including email, SMS, and HTTP(S). You can also use SNS to send messages to AWS Lambda functions.

- In SNS every topic has an ARN (Amazon Resource Name) that is used to identify the topic.

In [5]:
# Initialize boto3 client for SNS
sns = boto3.client('sns' , 
                   region_name='eu-central-1',
                   aws_access_key_id=f'{ACCESS_KEY_ID}',
                   aws_secret_access_key=f'{AWS_SECRET_KEY}')

In [6]:
# Create the city_alerts topic
response = sns.create_topic(Name='city_alerts')
c_alerts_arn = response['TopicArn']

In [7]:
# Re-create the city_alerts topic using a oneliner
c_alerts_arn_1 = sns.create_topic(Name='city_alerts')['TopicArn']

# Compare the two to make sure they match
print(c_alerts_arn == c_alerts_arn_1)

True


In [45]:
# Creating multiple topics
# Create list of departments
departments = ['trash', 'streets', 'water']

for dept in departments:
  	# For every department, create a general topic
    sns.create_topic(Name="{}_general".format(dept))
    
    # For every department, create a critical topic
    sns.create_topic(Name="{}_critical".format(dept))

# Print all the topics in SNS
response = sns.list_topics()
print(response['Topics'])

[{'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_general'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_notifications'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:trash_critical'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:trash_general'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:trash_notifications'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:water_critical'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:water_general'}]


In [46]:
# Get the current list of topics
topics = sns.list_topics()['Topics']

for topic in topics:
  # For each topic, if it is not marked critical, delete it
  if "critical" not in topic['TopicArn']:
    sns.delete_topic(TopicArn=topic['TopicArn'])
    
# Print the list of remaining critical topics
print(sns.list_topics()['Topics'])

[{'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:trash_critical'}, {'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:water_critical'}]


### SNS Subscription

In [47]:
# Creating an SNS subscription
response = sns.subscribe(TopicArn = 'arn:aws:sns:eu-central-1:121678499034:streets_critical', 
                         Protocol = 'SMS', 
                         Endpoint = '+1766035489')

In [48]:
# Create an email subscription
response = sns.subscribe(TopicArn = 'arn:aws:sns:eu-central-1:121678499034:streets_critical', 
                         Protocol = 'email', 
                         Endpoint = 'ssamilozkan@gmail.com')
# The subscription is pending confirmation
# Confirm the subscription
# Once you confirm the subscription, you will receive a welcome email from Amazon SNS

In [49]:
response

{'SubscriptionArn': 'pending confirmation',
 'ResponseMetadata': {'RequestId': 'e35ea051-a6de-5b0b-9628-fbae844de23a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e35ea051-a6de-5b0b-9628-fbae844de23a',
   'content-type': 'text/xml',
   'content-length': '298',
   'date': 'Mon, 28 Nov 2022 20:55:46 GMT'},
  'RetryAttempts': 0}}

In [50]:
# List the subscriptions by topic
sns.list_subscriptions_by_topic(TopicArn='arn:aws:sns:eu-central-1:121678499034:streets_critical')

{'Subscriptions': [{'SubscriptionArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical:e05277a5-6777-4567-9ce7-f3210ec3be6a',
   'Owner': '121678499034',
   'Protocol': 'email',
   'Endpoint': 'ssamilozkan@gmail.com',
   'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'},
  {'SubscriptionArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical:3928f8fd-b657-4d03-b6e0-dccb15835096',
   'Owner': '121678499034',
   'Protocol': 'sms',
   'Endpoint': '+1766035489',
   'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'}],
 'ResponseMetadata': {'RequestId': 'd7ac7ba8-0e97-5c6a-9415-20572c02855b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd7ac7ba8-0e97-5c6a-9415-20572c02855b',
   'content-type': 'text/xml',
   'content-length': '1071',
   'date': 'Mon, 28 Nov 2022 20:55:48 GMT'},
  'RetryAttempts': 0}}

In [51]:
# Listing subscriptions
# This will give us back the same response, but for all topics, instead of just one that we specify.
sns.list_subscriptions()

{'Subscriptions': [],
 'NextToken': 'AAHLeQCFA+vg0KnRFnKKqa0oKf/0VzNp1n1TydNqVuhgfw==',
 'ResponseMetadata': {'RequestId': '02377749-6d64-5c1d-af45-2cb64ff2352c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '02377749-6d64-5c1d-af45-2cb64ff2352c',
   'content-type': 'text/xml',
   'content-length': '367',
   'date': 'Mon, 28 Nov 2022 20:55:49 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# Deleting a subscription
sns.unsubscribe(SubscriptionArn='arn:aws:sns:eu-central-1:121678499034:streets_critical')

In [53]:
# What if we choose to stop sending notifications via SMS?

# Get the list of subscriptions
response = sns.list_subscriptions_by_topic(TopicArn='arn:aws:sns:eu-central-1:121678499034:streets_critical')
subs = response['Subscriptions']
subs

[{'SubscriptionArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical:e05277a5-6777-4567-9ce7-f3210ec3be6a',
  'Owner': '121678499034',
  'Protocol': 'email',
  'Endpoint': 'ssamilozkan@gmail.com',
  'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'},
 {'SubscriptionArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical:3928f8fd-b657-4d03-b6e0-dccb15835096',
  'Owner': '121678499034',
  'Protocol': 'sms',
  'Endpoint': '+1766035489',
  'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'}]

In [54]:
subs = pd.DataFrame(subs)
subs

,SubscriptionArn,Owner,Protocol,Endpoint,TopicArn
0,arn:aws:sns:eu-central-1:121678499034:streets_...,121678499034,email,ssamilozkan@gmail.com,arn:aws:sns:eu-central-1:121678499034:streets_...
1,arn:aws:sns:eu-central-1:121678499034:streets_...,121678499034,sms,+1766035489,arn:aws:sns:eu-central-1:121678499034:streets_...


In [ ]:
# Unsubscribe from all SMS subscriptions
for sub in subs:
  if sub['Protocol'] == 'SMS':
    sns.unsubscribe(sub['SubscriptionArn'])

In [56]:
esponse = sns.list_subscriptions_by_topic(TopicArn='arn:aws:sns:eu-central-1:121678499034:streets_critical')
subs = response['Subscriptions']
subs
# You can check the aws dashboard to see the changes

[{'SubscriptionArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical:e05277a5-6777-4567-9ce7-f3210ec3be6a',
  'Owner': '121678499034',
  'Protocol': 'email',
  'Endpoint': 'ssamilozkan@gmail.com',
  'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'},
 {'SubscriptionArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical:3928f8fd-b657-4d03-b6e0-dccb15835096',
  'Owner': '121678499034',
  'Protocol': 'sms',
  'Endpoint': '+1766035489',
  'TopicArn': 'arn:aws:sns:eu-central-1:121678499034:streets_critical'}]

### Sending a message

In [33]:
# Publishing to a Topic
# This will broadcast the message to every subscriber of the topic
response = sns.publish(TopicArn='arn:aws:sns:eu-central-1:121678499034:streets_critical', 
                       Message='The trash is not being picked up on time!', # Message
                       Subject = 'Trash Alert for Email') # Subject

In [40]:
# Sending a custom message
num_of_reports = 10
response = sns.publish(TopicArn='arn:aws:sns:eu-central-1:121678499034:streets_critical', 
                          Message = 'There are {} reports outstanding'.format(num_of_reports),
                          Subject = 'Trash Alert for Email')

In [38]:
# Sending a singel SMS
# For this method we don't have to worry about having a topic or subscribers to it.
# To send a single SMS, we also use the boto3 SNS client's publish method.
response = sns.publish(PhoneNumber='+4917660354789',
                       Message='The trash is not being picked up on time!')


## Building a notification System



In [57]:
# Initialize boto3 client for SNS
sns = boto3.client('sns' , 
                   region_name='eu-central-1',
                   aws_access_key_id=f'{ACCESS_KEY_ID}',
                   aws_secret_access_key=f'{AWS_SECRET_KEY}')

In [58]:
# Create topic and store their ARNs
trash_arn = sns.create_topic(Name='trash_notifications')['TopicArn']
streets_arn = sns.create_topic(Name='streets_notifications')['TopicArn']

In [81]:
# upload data for example
s3.upload_file(Filename= '/Users/ssamilozkan/Desktop/aws_boto/dataset/contacts.csv' ,
               Bucket= 'my-first-bucket-128341164379' , 
               Key= 'contacts.csv', 
               ExtraArgs={'ContentType': 'text/csv', 'ACL': 'public-read'}) 

In [90]:
# upload data for example
s3.upload_file(Filename= '/Users/ssamilozkan/Desktop/aws_boto/dataset/final_report.csv' ,
               Bucket= 'my-first-bucket-128341164379' , 
               Key= 'final_report.csv', 
               ExtraArgs={'ContentType': 'text/csv', 'ACL': 'public-read'}) 

In [74]:
contacts = pd.read_csv('https://my-first-bucket-128341164379.s3.eu-central-1.amazonaws.com/contacts.csv')
contacts

,name,email,Phone,Department
0,John,js@fake.com,123-456-7890,trash
1,Jane,fan@fake.com,123-456-7890,trash
2,Joe,who@fake.com,123-456-7890,streets
3,Jim,who2@fake.com,123-456-7890,streets
4,Janex,who3@fake.com,123-456-7890,streets


In [77]:
# Subscriber users to the topics
def subscribe_user(user_row):
    if user_row['Department'] == 'trash':
        sns.subscribe(TopicArn=trash_arn,Protocol='email',Endpoint=(user_row['email']))
        sns.subscribe(TopicArn=trash_arn,Protocol='SMS',Endpoint=str(user_row['Phone']))
    else:
        sns.subscribe(TopicArn=streets_arn,Protocol='email',Endpoint=(user_row['email']))
        sns.subscribe(TopicArn=streets_arn,Protocol='SMS',Endpoint=str(user_row['Phone']))

In [78]:
#Apply the subscribe_user metehod to every row 
# The users are subscribed to the topics based on their department
# you can check the aws dashboard to see the changes
# For the email subscription, you will receive a confirmation email
contacts.apply(subscribe_user,axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [95]:
# Get the aggregated numnbers
# Now we need to get last month's aggregated report.
# Because it's public we can simply use pd to load report into a dataframe
df = pd.read_csv('https://my-first-bucket-128341164379.s3.eu-central-1.amazonaws.com/final_report.csv') 
df

,service_name,request_count
0,Illegal Dumping,2580
1,Potential Missed Collection,150
2,Pothole,1170
3,Trafficc Sign - Maintenance,210
4,Traffic Sign - Repair,100
5,Traffic Sign - Replace,190


In [96]:
# We set the index on service_name for easier access.
df.set_index('service_name',inplace=True)

In [97]:
df

,request_count
service_name,
Illegal Dumping,2580
Potential Missed Collection,150
Pothole,1170
Trafficc Sign - Maintenance,210
Traffic Sign - Repair,100
Traffic Sign - Replace,190


In [98]:
# Get the aggreated numbers
trash_violations_count = df.at['Illegal Dumping','request_count']
streets_violations_count = df.at['Pothole','request_count']

In [99]:
# Send Alerts
if trash_violations_count > 100:
    sns.publish(TopicArn=trash_arn,
                Message='There are {} trash violations'.format(trash_violations_count),
                Subject='Trash Alert')
    
if streets_violations_count > 30:
    sns.publish(TopicArn=streets_arn, 
                Message='There are {} streets violations'.format(streets_violations_count),
                Subject='Streets Alert')